In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [32]:
df = pd.read_csv('Preprocessed_Data.csv')

In [33]:
df

,Sentence,Sentiment,text_length,cleaned_text,tokenized_text,word_lengths,document_length,bigrams,trigrams,avg_word_length
0,The GeoSolutions technology will leverage Bene...,positive,218,geosolutions technology leverage benefon gps s...,"['geosolutions', 'technology', 'leverage', 'be...","[12, 10, 8, 7, 3, 8, 9, 8, 5, 6, 10, 9, 8, 8, ...",21,"[('geosolutions', 'technology'), ('technology'...","[('geosolutions', 'technology', 'leverage'), (...",7.714286
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,55,esi low 1 50 2 50 bk real possibility,"['esi', 'low', '1', '50', '2', '50', 'bk', 're...","[3, 3, 1, 2, 1, 2, 2, 4, 11]",9,"[('esi', 'low'), ('low', '1'), ('1', '50'), ('...","[('esi', 'low', '1'), ('low', '1', '50'), ('1'...",3.222222
2,"For the last quarter of 2010 , Componenta 's n...",positive,193,last quarter 2010 componenta net sale doubled ...,"['last', 'quarter', '2010', 'componenta', 'net...","[4, 7, 4, 10, 3, 4, 7, 7, 6, 6, 4, 7, 5, 4, 3,...",21,"[('last', 'quarter'), ('quarter', '2010'), ('2...","[('last', 'quarter', '2010'), ('quarter', '201...",5.000000
3,According to the Finnish-Russian Chamber of Co...,neutral,128,according finnish russian chamber commerce maj...,"['according', 'finnish', 'russian', 'chamber',...","[9, 7, 7, 7, 8, 5, 12, 7, 7, 9, 6]",11,"[('according', 'finnish'), ('finnish', 'russia...","[('according', 'finnish', 'russian'), ('finnis...",7.636364
4,The Swedish buyout firm has sold its remaining...,neutral,135,swedish buyout firm sold remaining 22 4 percen...,"['swedish', 'buyout', 'firm', 'sold', 'remaini...","[7, 6, 4, 4, 9, 2, 1, 7, 5, 6, 8, 5, 6, 7, 6, 7]",16,"[('swedish', 'buyout'), ('buyout', 'firm'), ('...","[('swedish', 'buyout', 'firm'), ('buyout', 'fi...",5.625000
...,...,...,...,...,...,...,...,...,...,...
5837,RISING costs have forced packaging producer Hu...,negative,107,rising cost forced packaging producer huhtamak...,"['rising', 'cost', 'forced', 'packaging', 'pro...","[6, 4, 6, 9, 8, 9, 3, 2, 3, 9, 13, 5]",12,"[('rising', 'cost'), ('cost', 'forced'), ('for...","[('rising', 'cost', 'forced'), ('cost', 'force...",6.416667
5838,Nordic Walking was first used as a summer trai...,neutral,83,nordic walking first used summer training meth...,"['nordic', 'walking', 'first', 'used', 'summer...","[6, 7, 5, 4, 6, 8, 6, 5, 7, 5]",10,"[('nordic', 'walking'), ('walking', 'first'), ...","[('nordic', 'walking', 'first'), ('walking', '...",5.900000
5839,"According shipping company Viking Line , the E...",neutral,99,according shipping company viking line eu deci...,"['according', 'shipping', 'company', 'viking',...","[9, 8, 7, 6, 4, 2, 8, 11, 9, 6]",10,"[('according', 'shipping'), ('shipping', 'comp...","[('according', 'shipping', 'company'), ('shipp...",7.000000
5840,"In the building and home improvement trade , s...",neutral,88,building home improvement trade sale decreased...,"['building', 'home', 'improvement', 'trade', '...","[8, 4, 11, 5, 4, 9, 2, 1, 3, 3, 1, 2]",12,"[('building', 'home'), ('home', 'improvement')...","[('building', 'home', 'improvement'), ('home',...",4.416667


In [34]:
df.dropna(inplace=True)

In [35]:
label_encoder = LabelEncoder()
df['Sentiment_encoded'] = label_encoder.fit_transform(df['Sentiment'])
x = df['cleaned_text']
y = df['Sentiment_encoded']

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [37]:
tfidf = TfidfVectorizer(max_features=10000)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

In [38]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [39]:
y_train_cat

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [52]:
def create_dense_model(input_dim, num_classes):
    model = Sequential()
    model.add(Dense(512, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [53]:
input_dim = x_train_tfidf.shape[1]
num_classes = y_train_cat.shape[1]
model = create_dense_model(input_dim, num_classes)

In [54]:
history = model.fit(
    x_train_tfidf.toarray(), y_train_cat,
    epochs=10,
    batch_size=32,
    verbose=1,
    validation_data=(x_test_tfidf.toarray(), y_test_cat)
)

Epoch 1/10
146/146 [==============================] - 11s 71ms/step - loss: 0.8584 - accuracy: 0.6042 - val_loss: 0.7129 - val_accuracy: 0.6963
Epoch 2/10
146/146 [==============================] - 10s 68ms/step - loss: 0.4303 - accuracy: 0.7990 - val_loss: 0.7372 - val_accuracy: 0.6604
Epoch 3/10
146/146 [==============================] - 10s 68ms/step - loss: 0.2114 - accuracy: 0.8889 - val_loss: 0.9809 - val_accuracy: 0.6527
Epoch 4/10
146/146 [==============================] - 10s 68ms/step - loss: 0.1560 - accuracy: 0.8981 - val_loss: 1.1495 - val_accuracy: 0.6476
Epoch 5/10
146/146 [==============================] - 10s 69ms/step - loss: 0.1312 - accuracy: 0.9176 - val_loss: 1.2746 - val_accuracy: 0.6595
Epoch 6/10
146/146 [==============================] - 10s 69ms/step - loss: 0.1207 - accuracy: 0.9159 - val_loss: 1.4457 - val_accuracy: 0.6536
Epoch 7/10
146/146 [==============================] - 10s 67ms/step - loss: 0.1145 - accuracy: 0.9189 - val_loss: 1.5179 - val_accuracy:

In [56]:
y_pred_prob = model.predict(x_test_tfidf.toarray())
y_pred = np.argmax(y_pred_prob, axis=1)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

37/37 [==============================] - 0s 6ms/step
              precision    recall  f1-score   support

    negative       0.20      0.17      0.18       172
     neutral       0.70      0.76      0.73       626
    positive       0.75      0.69      0.72       371

    accuracy                           0.65      1169
   macro avg       0.55      0.54      0.54      1169
weighted avg       0.64      0.65      0.64      1169

